# Zakupki website scraping for Piotr

The aim of this notebook is to scrape details of each contract hosted on the Russian Zakupki public sector contract awarding website.

The input for this project will be the Zakupki URL. This code can be run on different dates to pull fresh contract data.

Method:
1.   Identify the number of pages of contracts to be scraped (using the contract filters provided).
2.   Iterate through each page, scraping the registration number of each contract.
3.   Access the website for each contract by placing the registraion number in the URL.
4.   Scrape the details for each contract and add them to a list of Contracts dataclasses.
5.   Format these Contract objects as a dataframe and output the dataframe to a csv file.


The output of this project will be the CSV file, with each row representing a new contract from the webstie.


### Section 1: Setup

In [3]:
import requests
from bs4 import BeautifulSoup
from datetime import date
from dataclasses import dataclass
from tqdm import tqdm
from dateutil import parser
from threading import Thread
import pandas as pd
import datetime
from datetime import date, timedelta
import logging
import http.client



In [4]:
logging = False

if logging:

    http.client.HTTPConnection.debuglevel = 1

    # You must initialize logging, otherwise you'll not see debug output.
    logging.basicConfig()
    logging.getLogger().setLevel(logging.DEBUG)
    requests_log = logging.getLogger("requests.packages.urllib3")
    requests_log.setLevel(logging.DEBUG)
    requests_log.propagate = True

### Section 2: Determine Number of pages to scrape
Test connection to the website and determine number of pages to scrape

In [6]:
# Getting the dates we want to scrape.


url="https://zakupki.gov.ru/epz/order/extendedsearch/results.html?searchString=&morphology=on&search-filter=%D0%94%D0%B0%D1%82%D0%B5+%D1%80%D0%B0%D0%B7%D0%BC%D0%B5%D1%89%D0%B5%D0%BD%D0%B8%D1%8F&pageNumber={}&sortDirection=false&recordsPerPage=_500&showLotsInfoHidden=false&savedSearchSettingsIdHidden=&sortBy=UPDATE_DATE&fz44=on&pc=on&placingWayList=&selectedLaws=FZ44&priceFromGeneral=10000000&priceFromGWS=&priceFromUnitGWS=&priceToGeneral=&priceToGWS=&priceToUnitGWS=&currencyIdGeneral=-1&publishDateFrom={}&publishDateTo={}&applSubmissionCloseDateFrom=&applSubmissionCloseDateTo=&customerIdOrg=&customerFz94id=&customerTitle=&okpd2Ids=&okpd2IdsCodes="
start = date(2022, 5, 2)
end = date(2022, 5, 20)
days = timedelta(days=1)

calendar=[]

### this actually works, set start & end date, it creates search results urls for each days with some criteria to limit the number (10 mln RUB, finished)
while start<=end:
  # tempURL=url.format(start.strftime('%d.%m.%Y'),start.strftime('%d.%m.%Y'))
  calendar.append(start.strftime('%d.%m.%Y'))
  start+=days

print(calendar)
  






['02.05.2022', '03.05.2022', '04.05.2022', '05.05.2022', '06.05.2022', '07.05.2022', '08.05.2022', '09.05.2022', '10.05.2022', '11.05.2022', '12.05.2022', '13.05.2022', '14.05.2022', '15.05.2022', '16.05.2022', '17.05.2022', '18.05.2022', '19.05.2022', '20.05.2022']



### Section 3: Scrape each registration number

Scrape the reg numbers of each contract, so they can be accessed individually

In [7]:
def getPage(tempURL):
  # If User-Agent is not set to custom, the website will know a Python script is accessing it and block some of the request
  return BeautifulSoup(requests.get(tempURL, headers={'User-Agent': 'Custom'}).content, "html.parser")

In [8]:
class Page:

    def __init__(self, day, pageNum, pagefile):

        self.day = day
        self.pageNum = pageNum
        self.pagefile = pagefile


In [9]:
# Getting the web page for all the contracts for each date in the range we want to scrape.


pages = []


for day in calendar:

  tempURL = url.format(1, day, day)

  page = getPage(tempURL)

  # Scrape the max number of pages
  try:
    maxPageNum = int(page.select('a[data-pagenumber]')[-2].find("span").text)
  except:
    maxPageNum = 1

  print(maxPageNum)
    
  for i in range(1, maxPageNum+1):

    contracts = []

    # Creating a temporary URL for each page containing contracts
    tempPageURL = url.format(i, day, day)
    print(tempPageURL)

    # Request the page and format it as a BeautifulSoup object so that we can perform scrapings
    page = getPage(tempPageURL)

    pages.append(Page(day, i, page))

    # print(page)


    

print(str(len(pages)), " pages found")


1
https://zakupki.gov.ru/epz/order/extendedsearch/results.html?searchString=&morphology=on&search-filter=%D0%94%D0%B0%D1%82%D0%B5+%D1%80%D0%B0%D0%B7%D0%BC%D0%B5%D1%89%D0%B5%D0%BD%D0%B8%D1%8F&pageNumber=1&sortDirection=false&recordsPerPage=_500&showLotsInfoHidden=false&savedSearchSettingsIdHidden=&sortBy=UPDATE_DATE&fz44=on&pc=on&placingWayList=&selectedLaws=FZ44&priceFromGeneral=10000000&priceFromGWS=&priceFromUnitGWS=&priceToGeneral=&priceToGWS=&priceToUnitGWS=&currencyIdGeneral=-1&publishDateFrom=02.05.2022&publishDateTo=02.05.2022&applSubmissionCloseDateFrom=&applSubmissionCloseDateTo=&customerIdOrg=&customerFz94id=&customerTitle=&okpd2Ids=&okpd2IdsCodes=
1
https://zakupki.gov.ru/epz/order/extendedsearch/results.html?searchString=&morphology=on&search-filter=%D0%94%D0%B0%D1%82%D0%B5+%D1%80%D0%B0%D0%B7%D0%BC%D0%B5%D1%89%D0%B5%D0%BD%D0%B8%D1%8F&pageNumber=1&sortDirection=false&recordsPerPage=_500&showLotsInfoHidden=false&savedSearchSettingsIdHidden=&sortBy=UPDATE_DATE&fz44=on&pc=on&pl

In [21]:
regNumbers = []


for page in pages:


  # Obtain a list of all the sections of HTML containing a contract in the web page
  listOfContracts = page.pagefile.find_all("div", {"class": "registry-entry__header-mid__number"})
  

  # Segment the registration number from the URL of each contract
  for contract in listOfContracts:
    href = contract.find("a")['href']
    regNum = href[href.index("Number=")+7:]
    regNumbers.append(regNum)

  # print(regNumbers)

  print("done", i, len(listOfContracts), '\n')

done 1 0 

done 1 5 

done 1 310 

done 1 334 

done 1 388 

done 1 16 

done 1 15 

done 1 5 

done 1 5 

done 1 252 

done 1 260 

done 1 194 

done 1 14 

done 1 2 

done 1 112 

done 1 49 

done 1 0 

done 1 0 

done 1 0 



In [11]:
# url="https://zakupki.gov.ru/epz/contract/contractCard/common-info.html?reestrNumber=1270000031322000018"
# r=requests.get(url).text
# soup=BeautifulSoup(r)
# tables=soup.findAll('table')
# #Contracted=tables[3].find_all("td",{"class":"tableBlock__col"})  ### that's about finding the winner and winning bid, the real of the contract
# #Supplier=Contracted[2].text # WORKS
# #FinalPrice=Contracted[3].text  # works

# Bidders=tables[2].find_all("tr",{"class":"tableBlock__row"})
# #bidder1=Bidders[1].text
# #bid1=Bidders[2].text
# #bidder2=Bidders[3].text
# #bi2=Bidders[4].text
# print(Bidders)

########## try to identify the number of tableBlock__row? 1/2/3 etc. and then generate bid1 bidder1 accordingly?


### Section 4: Details scraping

The Contract Dataclass will store the information during scraping.
If any information can't be scraped, default values have been provided in their place

In [22]:
@dataclass
class Contract:

  # TODO: Add reg number to class

  # Main Section
  ID: str = "none"
  price: float = 0.0
  published: date = None
  deadline: date = None
  # organisationName: str = "none"

  # Tab 1
  method: str = "none"
  tenderObject: str = "none"
  organisationName: str = "none"
  address: str = "none"
  official: str = "none"
  productCode: str = "none"


  # Tab 2
  # We may be missing info for this tab
  participants: str = "none"
  bids: float = 0.0
  

Method for scraping the data from each contract

In [23]:
def scrapeData(reg):

  dir = "https://zakupki.gov.ru/epz/order/notice/ea44/view/{}.html?regNumber={}"


  # TODO Add function to retry contracts that have their connection aborted.


  # Getting the web page for the given contract
  tempDir = dir.format("common-info", reg)
  print(tempDir)
  page = getPage(tempDir)

  # print(page)

  ### Basic info - Unique ID, Initial price, publication date and deadline 

  # ID = page.find("span",{"class":"navBreadcrumb__text"}).text.strip()  ## Returns unique ID with No at the start from each particular tender page (not the results list)
  ID = reg
  try:
    price=page.find("span", {"class":"cardMainInfo__content cost"}).text.replace("\xa0",",").replace(" ","").replace("₽","").strip()
  except:
    price=0.0
  cardInfo=page.find_all("span",{"class":"cardMainInfo__content"}) ## there's list inside, with procurer and price if needed
  published=str(cardInfo[3].text).strip()
  deadline=str(cardInfo[4].text).strip()

  ### First part info - tender type, object, procurer, address and official in charge

  sectionInfo=page.find_all("span",{"class":"section__info"})       ## same issue, but we can get a lot of stuff from this tab
  method=sectionInfo[0].text
  tenderObject=sectionInfo[4].text
  organisationName=sectionInfo[8].text                              ## e.g. like that?
  address=str(sectionInfo[9].text).strip()                                       ## 6 and 7 positions on the list return the same data..?
  official=str(sectionInfo[11].text).strip()                                      ## just in case, price can be also extracted from here, nr. 13

  ### Data taken from purchase order, it might contain multiple codes, to be tested! 
  tableBlock=page.find_all("td",{"class":"tableBlock__col"})
  productCode=tableBlock[1].text.strip()

  ### Second part info - participants and bids


  tempDir = dir.format("supplier-results", reg)
  print(tempDir)
  page = getPage(tempDir)

  tableBlock2=page.find_all("td",{"class":"tableBlock__col"})
  participants=tableBlock2[2].text
  bids=tableBlock2[3].text

  print(participants, bids)

  # Create the Contract dataclass object and append it to a list of objects.
  # This method means that missing data can be accounted for.
  contract = Contract(ID, price, published, deadline, method, tenderObject, organisationName, address, official, productCode)
  contracts.append(contract)


### Section 5: Starting execution
Scrape the contracts themselves using threading

In [24]:
### this soup is used later on to extract values from tender page
#soup1 = BeautifulSoup(requests.get(url1, headers={'User-Agent': 'Custom'}).content, "html.parser")

In [25]:
from tqdm import tqdm

contracts = []


print("Starting scrape with {} reg numbers".format(len(regNumbers)))

# Ensuring that hte code works without threading first
scrapeData(regNumbers[0])

# print(contracts)

# for regNum in tqdm(regNumbers):
#     scrapeData(regNum)


# for regNumber in tqdm(regNumbers[:50]):
# # #  scrapeData(regNumber)
#   thread = Thread(target = scrapeData, args = (regNumber,))
#   thread.start()

# print(contracts)


# For each contract reg number, start a thread for its execution and display the progress.
# for reg in tqdm(regNumbers):
#   thread = Thread(target = scrapeData, args = (reg,))
#   thread.start()


# The contracts list now contains only successfully scraped contracts
#print(len(contracts))

Starting scrape with 1961 reg numbers
https://zakupki.gov.ru/epz/order/notice/ea44/view/common-info.html?regNumber=0869200000222001700
https://zakupki.gov.ru/epz/order/notice/ea44/view/supplier-results.html?regNumber=0869200000222001700

                                1 - Победитель
                                
                             
                                7 000 000,00
                            


Usually about 9/10 contracts are scraped successfully. This depends on how the Zakupki website is feeling that day. Sometimes erroneous webpages are returned instead of the webpage for the requested contract.

### Section 6: Output

Convert the list of contract classes to a dataframe so that they can be exported to a csv file

In [20]:
contract = contracts[0]
print(contract.__dict__.keys())
df = pd.DataFrame(columns=contract.__dict__.keys())

for contract in contracts:
  df = df.append(contract.__dict__, ignore_index=True)

with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(df)

dict_keys(['ID', 'price', 'published', 'deadline', 'method', 'tenderObject', 'organisationName', 'address', 'official', 'productCode', 'participants', 'bids'])
                     ID             price   published    deadline  \
0   0869200000222001700     11,400,000,00  03.05.2022  24.05.2022   
1   0869200000222001691     11,895,000,00  03.05.2022  20.05.2022   
2   0869200000222001695     18,219,924,00  03.05.2022  17.05.2022   
3   0128200000122002263     38,218,224,96  03.05.2022  19.05.2022   
4   0173200025422000017     34,149,635,18  04.05.2022  25.05.2022   
5   0134200000122002266     12,000,000,00  04.05.2022  25.05.2022   
6   2073400001222000013    907,766,833,94  04.05.2022  25.05.2022   
7   0809500000322001036     30,172,043,84  03.05.2022  16.05.2022   
8   0121200004722000358     24,754,762,50  04.05.2022  25.05.2022   
9   0848300051722000243     38,456,956,80  04.05.2022  25.05.2022   
10  0164200003022001265    153,300,000,00  04.05.2022  25.05.2022   
11  03481000

In [ ]:
df.to_csv("zakupki.csv")